In [2]:

%load_ext autoreload
%autoreload 2
import json, os, cv2
from pathlib import Path
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import xarray as xr
import seaborn as sns
import h5py as hf
from tqdm import tqdm
from tqdm import tqdm
import itertools 
from scipy.interpolate import interp1d
from scipy import signal
from matplotlib.backends.backend_pdf import PdfPages
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
import os, fnmatch
from scipy.spatial.distance import cdist
import matplotlib.gridspec as gridspec
from random import sample
from scipy.ndimage import gaussian_filter1d
from matplotlib.collections import LineCollection


import sys
sys.path.insert(0, 'C:/Users/nlab/Documents/GitHub/obstacle_avoidance')

from utils.base_functions import *
from src.utils.auxiliary import flatten_series
from src.utils.path import find
from src.base import BaseInput
from plots.plots import plot_oa


import warnings
warnings.filterwarnings('ignore')

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject
c:\Users\nlab\anaconda3\envs\ephys0\lib\site-packages\deeplabcut\__init__.py:78: UserWarning: 
        As PyTorch is not installed, unsupervised identity learning will not be available.
        Please run `pip install torch`, or ignore this warning.
        
  warnings.warn(


In [24]:
df = pd.read_hdf(r"D:\obstacle_avoidance\recordings\071323\G8CK1TT\oa\testG8CK1TT_071323_oa.h5")
pxl = pd.read_hdf(r"D:\obstacle_avoidance\recordings\071323\G8CKLT\oa\pxl_G8CKLT_071323_oa.h5")

In [25]:
pxl.obstacleBL_x

0     [315.3675842285156, 315.2027282714844, 315.378...
1     [315.38983154296875, 315.246337890625, 315.331...
2     [315.38153076171875, 315.2965087890625, 314.91...
3     [315.2753601074219, 314.71002197265625, 314.98...
4     [314.824951171875, 315.1207275390625, 314.9300...
                            ...                        
86    [277.7339782714844, 277.76171875, 277.86624145...
87    [282.241455078125, 281.95855712890625, 281.776...
88    [260.8709716796875, 260.786376953125, 260.6373...
89    [260.75152587890625, 261.2103271484375, 260.68...
90    [260.72802734375, 260.86273193359375, 260.6827...
Name: obstacleBL_x, Length: 91, dtype: object

In [30]:
pxl.arenaTR_x[0]

array([nan, 714.3243408203125, nan, nan, 714.5865478515625,
       714.3258056640625, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, 714.6538696289062, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, 714.6179809570312, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       715.075439453125, nan, 714.47119140625, nan, nan,
       714.5733032226562, nan, nan, nan, nan, 713.7300415039062,
       713.3570556640625, 713.1754760742188, 712.0828857421875,
       712.452392578125, 712.7825317382812, 713.8588256835938, nan, nan,
       nan, 714.8444213867188, 714.670166015625, 715.1302490234375, nan,
       nan, nan, nan, nan], dtype=object)

In [31]:
np.nanmedian(pxl.arenaTR_x[0],0) 

714.3984985351562

In [17]:
df.obstacleBL_x

0      [447.1151123046875, 447.0396423339844, 447.025...
1      [447.07421875, 447.0856018066406, 447.05786132...
2      [nan, nan, nan, nan, nan, nan, nan, nan, nan, ...
3      [270.88330078125, 271.0210876464844, 270.66171...
4      [271.36358642578125, 270.690185546875, 271.255...
                             ...                        
139    [272.0767822265625, 272.3780517578125, 271.348...
140    [267.3695983886719, 267.2640686035156, 267.503...
141    [272.1219482421875, 272.1671447753906, 271.933...
142    [449.205322265625, 449.25250244140625, 449.086...
143    [448.7060241699219, 448.6969299316406, 448.432...
Name: obstacleBL_x, Length: 144, dtype: object

In [14]:
df

,first_poke,second_poke,trial_timestamps,trial_vidframes,nose_x,nose_y,nose_likelihood,leftear_x,leftear_y,leftear_likelihood,...,obstacleBL_x_cm,obstacleBL_y_cm,leftportT_x_cm,leftportT_y_cm,leftportB_x_cm,leftportB_y_cm,rightportT_x_cm,rightportT_y_cm,rightportB_x_cm,rightportB_y_cm
0,44804.431283,44806.504076,"[44804.435814, 44804.452979, 44804.469081, 448...","[601, 602, 603, 604, 605, 606, 607, 608, 609, ...","[90.75823211669922, 90.13227081298828, 90.3419...","[259.46978759765625, 259.5683898925781, 259.61...","[0.9999994039535522, 0.9999991655349731, 0.999...","[109.61051177978516, 108.2193832397461, 107.59...","[255.2165985107422, 255.19593811035156, 255.66...","[0.9999995231628418, 0.9999998807907104, 0.999...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
1,44806.504076,44809.887628,"[44806.504166, 44806.519718, 44806.536755, 448...","[[725, 726, 727, 728, 729, 730, 731, 732, 733,...","[671.7935791015625, 672.3903198242188, 673.272...","[266.02532958984375, 265.1124572753906, 264.67...","[0.9999903440475464, 0.9999899864196777, 0.999...","[662.0313110351562, 662.109130859375, 662.6464...","[252.5039825439453, 252.08631896972656, 252.05...","[0.9999978542327881, 0.9999963045120239, 0.999...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
2,44809.887628,44812.763238,"[44809.905049, 44809.921049, 44809.937587, 448...","[929, 930, 931, 932, 933, 934, 935, 936, 937, ...","[90.0763931274414, 90.00740814208984, 89.26874...","[259.6579284667969, 259.7824401855469, 259.505...","[0.9999996423721313, 0.9999997615814209, 0.999...","[104.97773742675781, 104.67391967773438, 103.8...","[255.7764892578125, 256.252197265625, 256.0703...","[0.9999998807907104, 0.9999998807907104, 1.0, ...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ..."
3,44812.763238,44815.233292,"[44812.772057, 44812.788825, 44812.805388, 448...","[[1101, 1102, 1103, 1104, 1105, 1106, 1107, 11...","[670.5297241210938, 671.4808349609375, 672.303...","[264.4493408203125, 265.2351379394531, 265.607...","[0.9950258135795593, 0.9939115047454834, 0.992...","[657.111328125, 659.3954467773438, 663.9580078...","[262.4812316894531, 262.2870788574219, 258.837...","[0.9967846870422363, 0.9933741688728333, 0.997...",...,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...

In [8]:
port_arena_list = list_columns(df,['arena','leftportT','rightportT']) ## add left right port B and
port_arena_list = [i for i in port_arena_list if 'cm' in i]
for pos in port_arena_list:
   for ind,row in df.iterrows():
        df.at[ind,pos] = np.mean(row[pos])
port_arena_list = list_columns(df,['arena','leftportT','rightportT'])
port_arena_list = [i for i in port_arena_list if 'cm' in i]
for pos in port_arena_list:
    df[pos] = np.mean(df[pos])
           

In [9]:
df

,first_poke,second_poke,trial_timestamps,trial_vidframes,nose_x,nose_y,nose_likelihood,leftear_x,leftear_y,leftear_likelihood,...,rightportT_y,rightportT_likelihood,rightportB_x,rightportB_y,rightportB_likelihood,len,animal,date,task,condition
0,44804.431283,44806.504076,"[44804.435814, 44804.452979, 44804.469081, 448...","[601, 602, 603, 604, 605, 606, 607, 608, 609, ...","[90.75823211669922, 90.13227081298828, 90.3419...","[259.46978759765625, 259.5683898925781, 259.61...","[0.9999994039535522, 0.9999991655349731, 0.999...","[109.61051177978516, 108.2193832397461, 107.59...","[255.2165985107422, 255.19593811035156, 255.66...","[0.9999995231628418, 0.9999998807907104, 0.999...",...,"[259.5006103515625, 259.5478515625, 259.872100...","[0.9999998807907104, 0.9999997615814209, 0.999...","[662.7114868164062, 662.4907836914062, 662.765...","[260.9520263671875, 261.15850830078125, 261.53...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.99999988...",123.0,G8CK1TT,071323,oa,whiteblack
1,44806.504076,44809.887628,"[44806.504166, 44806.519718, 44806.536755, 448...","[[725, 726, 727, 728, 729, 730, 731, 732, 733,...","[671.7935791015625, 672.3903198242188, 673.272...","[266.02532958984375, 265.1124572753906, 264.67...","[0.9999903440475464, 0.9999899864196777, 0.999...","[662.0313110351562, 662.109130859375, 662.6464...","[252.5039825439453, 252.08631896972656, 252.05...","[0.9999978542327881, 0.9999963045120239, 0.999...",...,"[261.57403564453125, 261.3713684082031, 261.69...","[0.999998927116394, 0.9999990463256836, 0.9999...","[666.9411010742188, 667.0116577148438, 667.558...","[260.49212646484375, 260.15960693359375, 260.6...","[0.9999916553497314, 0.9999960660934448, 0.999...",203.0,G8CK1TT,071323,oa,whiteblack
2,44809.887628,44812.763238,"[44809.905049, 44809.921049, 44809.937587, 448...","[929, 930, 931, 932, 933, 934, 935, 936, 937, ...","[90.0763931274414, 90.00740814208984, 89.26874...","[259.6579284667969, 259.7824401855469, 259.505...","[0.9999996423721313, 0.9999997615814209, 0.999...","[104.97773742675781, 104.67391967773438, 103.8...","[255.7764892578125, 256.252197265625, 256.0703...","[0.9999998807907104, 0.9999998807907104, 1.0, ...",...,"[260.60540771484375, 260.1939392089844, 259.87...","[0.9999998807907104, 0.9999997615814209, 0.999...","[662.7360229492188, 662.79248046875, 662.87390...","[262.31292724609375, 261.7840881347656, 261.62...","[1.0, 1.0, 1.0, 0.9999998807907104, 0.99999988...",171.0,G8CK1TT,071323,oa,whiteblack
3,44812.763238,44815.233292,"[44812.772057, 44812.788825, 44812.805388, 448...","[[1101, 1102, 1103, 1104, 1105, 1106, 1107, 11...","[670.5297241210938, 671.4808349609375, 672.303...","[264.4493408203125, 265.2351379394531, 265.607...","[0.9950258135795593, 0.9939115047454834, 0.992...","[657.111328125, 659.3954467773438, 663.9580078...","[262.4812316894531, 262.2870788574219, 258.837...","[0.9967846870422363, 0.9933741688728333, 0.997...",...,"[262.7078552246094, 262.3763427734375, 262.007...","[0.9999866485595703, 0.9999947547912598, 0.999...","[668.6475219726562, 669.180908203125, 669.3231...","[262.0871887207031, 262.02911376953125, 261.76...","[0.9999537467956543, 0.9999680519104004, 0.999...",147.0,G8CK1TT,071323,oa,whiteblack
4,44815.233292,44818.215475,"[44815.239475, 44815.256204, 44815.272998, 448...","[1249, 1250, 1251, 1252, 1253, 1254, 1255, 125...","[95.31839752197266, 94.98600769042969, 94.6304...","[259.3034362792969, 259.1989440917969, 259.498...","[0.9987138509750366, 0.9973199963569641, 0.989...","[101.03604888916016, 100.696533203125, 99.8996...","[263.4975891113281, 263.67059326171875, 263.78...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",...,"[260.2019348144531, 260.3462219238281, 260.493...","[0.9999997615814209, 0.9999996423721313, 0.999...","[662.7063598632812, 662.6775512695312, 662.616...","[261.54510498046875, 261.8872375488281, 262.00...","[0.9999998807907104, 0.9999998807907104, 0.999...",178.0,G8CK1TT,071323,oa,whiteblack
...,...,...,...,...,...,...,...,...,...,.

In [10]:
df.condition

0      whiteblack
1      whiteblack
2      whiteblack
3      whiteblack
4      whiteblack
          ...    
113    whiteblack
114    whiteblack
115    whiteblack
116    whiteblack
117    whiteblack
Name: condition, Length: 118, dtype: object

In [10]:
df.columns

Index(['first_poke', 'second_poke', 'trial_timestamps', 'trial_vidframes',
       'nose_x', 'nose_y', 'nose_likelihood', 'leftear_x', 'leftear_y',
       'leftear_likelihood', 'rightear_x', 'rightear_y', 'rightear_likelihood',
       'spine_x', 'spine_y', 'spine_likelihood', 'midspine_x', 'midspine_y',
       'midspine_likelihood', 'tailbase_x', 'tailbase_y',
       'tailbase_likelihood', 'midtail_x', 'midtail_y', 'midtail_likelihood',
       'tailend_x', 'tailend_y', 'tailend_likelihood', 'arenaTL_x',
       'arenaTL_y', 'arenaTL_likelihood', 'arenaTR_x', 'arenaTR_y',
       'arenaTR_likelihood', 'arenaBL_x', 'arenaBL_y', 'arenaBL_likelihood',
       'arenaBR_x', 'arenaBR_y', 'arenaBR_likelihood', 'obstacleTL_x',
       'obstacleTL_y', 'obstacleTL_likelihood', 'obstacleTR_x', 'obstacleTR_y',
       'obstacleTR_likelihood', 'obstacleBR_x', 'obstacleBR_y',
       'obstacleBR_likelihood', 'obstacleBL_x', 'obstacleBL_y',
       'obstacleBL_likelihood', 'leftportT_x', 'leftportT_y',
      

In [11]:
df.head()

,first_poke,second_poke,trial_timestamps,trial_vidframes,nose_x,nose_y,nose_likelihood,leftear_x,leftear_y,leftear_likelihood,...,rightportT_y,rightportT_likelihood,rightportB_x,rightportB_y,rightportB_likelihood,len,animal,date,task,condition
0,46850.591321,46857.608204,"[46850.595276, 46850.61175, 46850.628518, 4685...","[77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 8...","[nan, 632.8485717773438, 631.9840087890625, 63...","[nan, 163.82443237304688, 163.44210815429688, ...","[0.5765019059181213, 0.9487873911857605, 0.994...","[642.82568359375, 636.9893798828125, 634.43121...","[186.78201293945312, 186.78277587890625, 183.2...","[0.9921253323554993, 0.9927536845207214, 0.993...",...,"[270.9154968261719, 271.0943298339844, 270.999...","[0.9997239708900452, 0.9996827840805054, 0.999...","[673.9995727539062, 673.6875, 674.055603027343...","[271.037353515625, 271.0587463378906, 271.0313...","[0.9999963045120239, 0.9999954700469971, 0.999...",420.0,G8CKLN,071023,oadark,whiteblack
1,46857.608204,46859.959577,"[46857.614643, 46857.630784, 46857.647475, 468...","[[498, 499, 500, 501, 502, 503, 504, 505, 506,...","[676.5250854492188, 678.0358276367188, nan, na...","[278.6856994628906, 278.7580261230469, nan, na...","[0.8860661387443542, 0.8868556618690491, 0.740...","[675.62841796875, 675.9398193359375, 676.51184...","[262.55126953125, 262.1081848144531, 262.48379...","[0.9999994039535522, 0.9999991655349731, 0.999...",...,"[269.4980773925781, 269.50177001953125, 269.82...","[0.9999276399612427, 0.999923825263977, 0.9999...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.1419605016708374, 0.09557054936885834, 0.18...",140.0,G8CKLN,071023,oadark,whiteblack
2,46859.959577,46862.298611,"[46859.965427, 46859.981964, 46859.998272, 468...","[639, 640, 641, 642, 643, 644, 645, 646, 647, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[0.7011783719062805, 0.686337947845459, 0.6586...","[101.36710357666016, 100.48766326904297, 100.2...","[272.9229431152344, 272.5069580078125, 272.713...","[0.9999992847442627, 0.9999986886978149, 0.999...",...,"[272.90802001953125, 272.82220458984375, 273.1...","[0.9997106194496155, 0.9998188614845276, 0.999...","[670.2900390625, 669.9945678710938, 669.696899...","[273.7950744628906, 274.1410827636719, 273.933...","[0.9999903440475464, 0.999995231628418, 0.9999...",139.0,G8CKLN,071023,oadark,whiteblack
3,46862.298611,46865.587827,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,G8CKLN,071023,oadark,whiteblack
4,46865.587827,46868.496332,"[46865.600716, 46865.61678, 46865.6336, 46865....","[977, 978, 979, 980, 981, 982, 983, 984, 985, ...","[96.5356674194336, 96.68403625488281, 96.20884...","[259.2501525878906, 259.5872497558594, 258.914...","[0.982207179069519, 0.8784804344177246, 0.9686...","[103.33856201171875, 103.03973388671875, 101.9...","[271.66937255859375, 271.9025573730469, 270.21...","[0.9999998807907104, 0.9999996423721313, 0.999...",...,"[273.232666015625, 272.68896484375, 273.390441...","[0.9997456669807434, 0.9997995495796204, 0.999...","[670.1148071289062, 669.794677734375, 670.1492...","[273.9629211425781, 273.861572265625, 274.6862...","[0.9999936819076538, 0.9999951124191284, 0.999...",173.0,G8CKLN,071023,oadark,whiteblack


In [25]:
import random
random.randint(0,1)

1

In [30]:
def process(value):
  prob = .7 
  if value == True:
   if random.random() < prob:
    return 1
   else:
    return 0

  

In [39]:
process(True)

1

In [44]:
random.random()



0.5132446030939827

In [4]:
time_thresh = df['len'].quantile(0.9)
df['time'] = df['len']/60
df = df[df['len']<time_thresh]
session = plot_oa('empty',df)
session.single_day_df('obstacle',6)
session.plot_trace_cluster_single_animal()

KeyError: 'start'